# Documentação da Solução - Busca Semântica com FastText

Este notebook documenta a solução desenvolvida para o case de Data Science focado em busca semântica de empresas usando FastText e técnicas de similaridade.

## Sumário
1. Introdução ao Problema
2. Configuração do Ambiente
3. Pré-processamento dos Dados
4. Modelo FastText
5. Sistema de Busca
6. Avaliação e Resultados

## 1. Introdução ao Problema

O desafio consiste em desenvolver um sistema de busca semântica que permita encontrar empresas a partir de texto livre digitado pelo usuário. O sistema deve:
- Lidar com variações de escrita e erros de digitação
- Considerar razão social e nome fantasia
- Usar informações contextuais como UF
- Retornar empresas ordenadas por similaridade

As métricas de avaliação são:
- Top-1: Percentual de acerto na primeira sugestão
- Top-5: Percentual de acerto entre as 5 primeiras sugestões

In [1]:
# Imports necessários
import pandas as pd
import numpy as np
import re
import unicodedata
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import FastText
from sklearn.metrics.pairwise import cosine_similarity
from fuzzywuzzy import fuzz
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

# Download recursos NLTK
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Nucleo Mundial de
[nltk_data]     Ne\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 2. Carregamento e Exploração dos Dados

O dataset contém informações de empresas com os seguintes campos:
- user_input: texto digitado pelo usuário
- uf: estado da empresa
- razaosocial: razão social oficial
- nome_fantasia: nome fantasia da empresa

In [2]:
# Carregamento dos dados
df = pd.read_parquet('dados/train.parquet')
df = df[['user_input', 'uf', 'razaosocial', 'nome_fantasia']].reset_index(drop=True)

print("Tamanho do dataset:", len(df))
print("\nPrimeiras linhas:")
df.head()

Tamanho do dataset: 255471

Primeiras linhas:


,user_input,uf,razaosocial,nome_fantasia
0,MAGAZINE L,SP,MAGAZINE LUIZA S/A,MAGAZINE LUIZA
1,PNEUS GP,PR,GP PNEUS LTDA,GP PNEUS
2,SANTA CRUZ DISTRIBUIDORA,RS,DISTRIBUIDORA DE MEDICAMENTOS SANTA CRUZ LTDA,SANTA CRUZ
3,DROGALL,SP,DROGAL FARMACEUTICA LTDA,DROGAL JAGUARIUNA
4,ESTAPAR BRASIL LTDA,ES,"ALLPARK EMPREENDIMENTOS, PARTICIPACOES E SERVI...",ESTAPAR


## 3. Pré-processamento dos Dados

### 3.1 Limpeza de Texto
Foi implementada a seguinte função de limpeza que:
- Remove acentos e caracteres especiais
- Converte para minúsculas
- Trata números como tokens especiais
- Remove stopwords específicas do domínio

In [11]:
def clean_text(text):
    """Pré-processamento otimizado para nomes de empresas"""
    text = str(text).lower().strip()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    
    # Remove caracteres especiais preservando números e alguns símbolos
    text = re.sub(r'[^a-z0-9\s&.-]', ' ', text)
    
    # Trata números como tokens especiais
    text = re.sub(r'(\d+)', r' \1 ', text)
    
    # Remove stopwords específicas do domínio
    stopwords = ['ltda.', 'me', 'epp', 'sa', 's/a', 'limitada', 'eireli']
    words = text.split()
    words = [w for w in words if w not in stopwords]
    
    return ' '.join(words)

# Exemplo
texto = "EMPRESA LTDA. & COMÉRCIO SA"
print(f"Original: {texto}")
print(f"Limpo: {clean_text(texto)}")

Original: EMPRESA LTDA. & COMÉRCIO SA
Limpo: empresa & comercio


### 3.2 Preparação dos Dados
Foi criado um campo combinando razão social e nome fantasia, com peso maior para o nome fantasia por ser mais próximo do que os usuários digitam.

In [12]:
def prepare_search_text(row):
    """Prepara texto para busca priorizando nome fantasia"""
    razao = clean_text(str(row.razaosocial)) if pd.notna(row.razaosocial) else ''
    fantasia = clean_text(str(row.nome_fantasia)) if pd.notna(row.nome_fantasia) else ''
    
    if razao and fantasia:        
        return f"{fantasia} {fantasia} {razao}"  # Duplica nome fantasia para dar mais peso
    
    return fantasia or razao

# Criação do campo target e split treino/teste
df['target_empresa'] = (df.razaosocial.fillna('') + ' ' + df.nome_fantasia.fillna('')).apply(clean_text)
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Aplica prepare_search_text
df_train['search_text'] = df_train.apply(prepare_search_text, axis=1)
df_test['search_text'] = df_test.apply(prepare_search_text, axis=1)

## 4. Modelo FastText

### 4.1 Tokenização e Treinamento
Foi implementada tokenização específica para empresas e treinado o modelo FastText.

In [13]:
def tokenize_business(text):
    """Tokenização específica para nomes de empresas"""
    text = clean_text(text)
    
    tokens = []
    for word in text.split():
        # Se tem número, mantém junto
        if any(c.isdigit() for c in word):
            tokens.append(word)
        else:
            # Mantém palavras pequenas (possíveis iniciais)
            if len(word) <= 2:
                tokens.append(word)
            else:
                tokens.extend(word_tokenize(word))
    return tokens

# Treina modelo FastText
corpus_ft = [tokenize_business(text) for text in df_train.search_text]

fasttext_model = FastText(
    sentences=corpus_ft,
    vector_size=300,     # Dimensão dos vetores
    window=5,            # Janela de contexto
    min_count=1,         # Inclui todas as palavras
    sg=1,               # Skip-gram
    negative=15,        # Negative sampling
    epochs=50,          # Número de épocas
    min_n=2,           # Tamanho mínimo de n-grams
    max_n=6            # Tamanho máximo de n-grams
)

### 4.2 Embeddings com Pesos Adaptativos
Foi implementada uma função que gera embeddings com pesos que se adaptam às características do texto:
- Maior peso para primeiras palavras
- Peso reduzido para tokens curtos
- Normalização do vetor final

In [14]:
def ft_embedding(text):
    """Gera embeddings com pesos adaptativos"""
    tokens = tokenize_business(text)
    vectors = []
    weights = []
    
    for i, token in enumerate(tokens):
        if token in fasttext_model.wv:
            vec = fasttext_model.wv[token]
            weight = 1.0
            
            # Primeiras palavras são mais importantes
            if i < 2:
                weight *= 1.2
            
            # Tokens curtos têm peso menor
            if len(token) <= 2:
                weight *= 0.6
                
            vectors.append(vec)
            weights.append(weight)
    
    if vectors:
        weights = np.array(weights).reshape(-1, 1)
        weighted_vectors = np.array(vectors) * weights
        emb = np.sum(weighted_vectors, axis=0) / np.sum(weights)
        return emb / np.linalg.norm(emb)
    
    return np.zeros(fasttext_model.vector_size)

# Gera embeddings para treino e teste
print("Gerando embeddings...")
df_train['ft_emb'] = df_train.search_text.apply(ft_embedding)
df_test['ft_emb'] = df_test.search_text.apply(ft_embedding)

Gerando embeddings...


## 5. Sistema de Busca

A busca é implementada usando uma função que:
- Filtra por UF quando informada
- Calcula similaridade de cosseno entre embeddings
- Retorna as k empresas mais similares

In [15]:
def get_topk(user_input, uf=None, k=5, base_busca=None):
    """Busca as k empresas mais similares"""
    texto = tokenize_business(user_input)
    data = base_busca.copy()
    
    if uf:
        data = data[data.uf == uf].reset_index(drop=True)
    
    emb = ft_embedding(texto)
    similaridade = cosine_similarity([emb], list(data.ft_emb.values))[0]
    
    top_indices = similaridade.argsort()[::-1][:k]
    return data.iloc[top_indices]

# Exemplo de busca
exemplo = get_topk('banco itau', uf='SP', base_busca=df_train)
print("\nExemplo de busca:")
print(exemplo[['razaosocial', 'nome_fantasia']])


Exemplo de busca:
              razaosocial   nome_fantasia
25799  ITAU UNIBANCO S.A.  BANCO ITAU S/A
29998  ITAU UNIBANCO S.A.  BANCO ITAU S/A
25014  ITAU UNIBANCO S.A.  BANCO ITAU S/A
47325  ITAU UNIBANCO S.A.  BANCO ITAU S/A
7707   ITAU UNIBANCO S.A.  BANCO ITAU S/A


## 6. Avaliação e Resultados

### 6.1 Função de Avaliação
Foi implementada uma função que avalia as métricas Top-1 e Top-5 no conjunto de teste.

In [16]:
def avaliar_modelo(df_teste, base_busca, batch_size=32):
    """Avalia as métricas Top-1 e Top-5"""
    total = len(df_teste)
    top1 = 0
    top5 = 0
    
    # Pré-processamento dos alvos
    alvos = df_teste.target_empresa.apply(clean_text).values
    
    with tqdm(total=total, desc='Avaliando modelo') as pbar:
        for i in range(0, total, batch_size):
            batch = df_teste.iloc[i:i+batch_size]
            
            for j, row in enumerate(batch.itertuples()):
                entrada = row.user_input
                uf = row.uf if hasattr(row, 'uf') else None
                
                resultados = get_topk(entrada, uf, k=5, base_busca=base_busca)
                pred_empresas = resultados.target_empresa.apply(clean_text).values
                
                if alvos[i+j] == pred_empresas[0]:
                    top1 += 1
                if alvos[i+j] in pred_empresas:
                    top5 += 1
                
                pbar.update(1)
    
    print(f'\nResultados:')
    print(f'Top-1: {top1/total:.2%}')
    print(f'Top-5: {top5/total:.2%}')
    
    return {'top1': top1/total, 'top5': top5/total}

### 6.2 Resultados

A avaliação do modelo no conjunto de teste mostra:
- Top-1 Accuracy: Porcentagem de vezes que a primeira sugestão está correta
- Top-5 Accuracy: Porcentagem de vezes que a resposta correta está entre as 5 primeiras sugestões

In [ ]:
# Avaliação final
print("Avaliando modelo no conjunto de teste...")
resultados = avaliar_modelo(df_test, df_train)

print("\nExemplos de busca:")
queries = [
    ("banco itau", "SP"),
    ("carrefour", "RJ"),
    ("casas bahia", None)
]

for query, uf in queries:
    print(f"\nBusca: {query} (UF: {uf if uf else 'Todos'})")
    results = get_topk(query, uf=uf, k=3, base_busca=df_train)
    print(results[['razaosocial', 'nome_fantasia']])

In [ ]:
# Exemplo de uso do sistema
queries = [
    ("banco itau", "SP"),
    ("carrefour", "RJ"),
    ("casas bahia", None)
]

print("Exemplos de busca:")
for query, uf in queries:
    print(f"\nBusca: {query} (UF: {uf if uf else 'Todos'})")
    resultados = get_topk(query, uf=uf, k=3, base_busca=df_train)
    print(resultados[['razaosocial', 'nome_fantasia']])